In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import skimage.transform as skt
from skimage.measure import compare_ssim  as ssim
import os
import gc
import pickle
import my_alg_grp
%matplotlib inline

In [5]:
#define loss function
def ave_cross_en( y_true, y_pred):
    loss = ( keras.losses.kullback_leibler_divergence( y_true, y_pred) +  keras.losses.kld( 1 - y_true, 1- y_pred))
    return loss
def ssim_loss(y_true, y_pred):
  return (1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0)))
def ave_cross_en_ssim( y_true, y_pred):
    loss = ( keras.losses.kullback_leibler_divergence( y_true, y_pred) +  keras.losses.kld( 1 - y_true, 1- y_pred) + (1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))))
    return loss
def ave_MAE_en( y_true, y_pred):
    loss = tf.keras.losses.MeanAbsoluteError(reduction="auto", name="mean_absolute_error")
    return loss

In [8]:
#define functions for calculating SSIM and MAE
def average_SSIM( y_true, y_pred):
    ssims = []
    y_true = np.squeeze( y_true)
    y_pred = np.squeeze( y_pred)

    if len( y_true) !=  len( y_pred):
        raise  NameError("length not match!")

    for i in range(len(y_pred)):
        x = y_pred[i]
        y = y_true[i]
        #ss = ssim(x, y,multichannel=True)
        ss = ssim(x, y)
        ssims.append(ss)
    ssims = np.array(ssims)
    ave_ssim = np.average(ssims)

    return ssims

def average_MAE( y_true, y_pred):
    mae = []
    y_true = np.squeeze( y_true)
    y_pred = np.squeeze( y_pred)

    if len( y_true) !=  len( y_pred):
        raise  NameError("length not match!")
    #a,b = y_true[1][0].shape
    a,b = y_true[1].shape
    for i in range(len(y_pred)):
        x = y_pred[i]
        y = y_true[i]
        d = abs(x-y)
        #m = sum(sum(sum(d)))/(3*128*128)
        m = sum(sum(d))/(a*b)
        mae.append(m)
    mae = np.array(mae)
    ave_mae = np.average(mae)

    return mae

def ave_SSIM( y_true, y_pred):
    #ssims = []
    y_true = np.squeeze( y_true)
    y_pred = np.squeeze( y_pred)
    ssim1 = tf.image.ssim(y_true, y_pred)
    #ss = ssim(y_true, y_pred)
    #ssims.append(ss)
    #ssims = np.array(ssims)
    ave_ssim = np.average(ssim1)

    return ave_ssim

In [11]:
#load dataset
mnist_x = np.load('I:/flow_test/20220902_20fps/ref_optrode_bead_for_flow_128_allnorm_ccm1_128.npy')
mnist_b = np.load('I:/flow_test/20220902_20fps/ccm_optrode_bead_for_flow_128_allnorm_ccm1_128.npy')

In [12]:
mnist_b = np.expand_dims( mnist_b, axis=3)
mnist_x = np.expand_dims( mnist_x, axis=3)

In [14]:
#seperate the dataset
x_test1 = mnist_x[:2000]
b_test1 = mnist_b[:2000]

x_eval1 = mnist_x[2000:4000]
b_eval1 = mnist_b[2000:4000]

x_train1 = mnist_x[4000:]
b_train1 = mnist_b[4000:]

In [19]:
#define model
model1 = build_model_128_filter_change1()
model1.compile(optimizer=keras.optimizers.Adam(1e-4, amsgrad= True), loss=ave_cross_en_ssim, metrics=['mae'])

In [26]:
#train model
model1.fit(b_train1, x_train1, batch_size=16, epochs=25, verbose=1,validation_data= ( b_eval1, x_eval1))

Train on 22088 samples, validate on 2000 samples
Epoch 1/25
22088/22088 [==============================] - 288s 13ms/sample - loss: 0.4323 - mean_absolute_error: 0.1026 - val_loss: 0.6624 - val_mean_absolute_error: 0.1434
Epoch 2/25
22088/22088 [==============================] - 289s 13ms/sample - loss: 0.3188 - mean_absolute_error: 0.0749 - val_loss: 0.4484 - val_mean_absolute_error: 0.0912
Epoch 3/25
22088/22088 [==============================] - 284s 13ms/sample - loss: 0.2844 - mean_absolute_error: 0.0682 - val_loss: 0.4633 - val_mean_absolute_error: 0.0950
Epoch 4/25
22088/22088 [==============================] - 284s 13ms/sample - loss: 0.2648 - mean_absolute_error: 0.0642 - val_loss: 0.3919 - val_mean_absolute_error: 0.0788
Epoch 5/25
22088/22088 [==============================] - 284s 13ms/sample - loss: 0.2510 - mean_absolute_error: 0.0615 - val_loss: 0.4159 - val_mean_absolute_error: 0.0882
Epoch 6/25
22088/22088 [==============================] - 284s 13ms/sample - loss: 0.2

In [22]:
#save model
model.save('I:/flow_test/20220902_20fps/optrode_1e4_bs16_epoch25_allnorm_ccm1_128')

In [27]:
#predict test data
y_pred1 = model1.predict(b_test1)

In [44]:
#show images
a = np.arange(0,20,1) 
test_fig = my_alg_grp.show_some_results( b_test1, x_test1, y_pred1,20, img_size=5, indexes = a, resize_shape=(128, 128))

In [47]:
#calculate ssim and mae
ssims_t = []
maes_t = []
ssims_tr = []
maes_tr = []
ssims_t  = average_SSIM(x_test1,y_pred1)
maes_t  = average_MAE(x_test1,y_pred1)

ave_ssims_t = np.average(ssims_t)
ave_maes_t = np.average(maes_t)

print(ave_ssims_t)
print(ave_maes_t)